# **Classification of the ModelNet dataset with ORION**


**Installation of the libraries**

In [1]:
#connect to Google drive to access datasets
from google.colab import drive
drive.mount('/content/drive')

#import libraries
import numpy as np
import torch
import h5py
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from torch.nn import CrossEntropyLoss


!pip install torchvision
from torchvision.transforms import Compose

#install wandb in order to obtain graphs
!pip install wandb
import wandb

# Check for GPU availability
device = "cuda" if torch.cuda.is_available() else "cpu"

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 10.3 MB/s eta 0:00:00



#**Data pre-processing**
**Functions**

In [2]:
class ImportDatas(Dataset):
  def __init__(self, file_txt_path, transform=None):

        with open(file_txt_path) as file:
            lines = file.readlines()
        #path to import the dataset
        self.paths = [r"/content/drive/MyDrive/ORION_dataset/ModelNet10_12rot/" + line.strip('\n')[40:] for line in lines]
        self.data, self.label, self.label_pose = self._combine_files()
        self.transform = transform

  def _combine_files(self):
    #extract objects and labels
        combined_data = None
        combined_label = None
        combined_label_pose = None

        for path in self.paths:
            file = h5py.File(path, 'r')
            data = file['data']

            label = np.int64(file['label'])
            label_pose = np.int64(file['label_pose'])

            if combined_data is None:
                combined_data = data
                combined_label = label
                combined_label_pose = label_pose
            else:
                combined_data = np.concatenate((combined_data, data), axis=0)
                combined_label = np.concatenate((combined_label, label), axis=0)
                combined_label_pose = np.concatenate((combined_label_pose, label_pose), axis=0)

        return combined_data, combined_label, combined_label_pose

  def __len__(self):
        return len(self.data)

  def __getitem__(self, idx):
        data = self.data[idx]
        label = self.label[idx]
        label_pose = self.label_pose[idx]
        if self.transform:
            data = self.transform(data)
        return data, label, label_pose


In [3]:
class CropTransform(torch.nn.Module):
  def forward(self, datas):
    #cropping datas from [36x36x36] to [32x32x32]
    cropped_datas = datas[: ,2:34, 2:34, 2:34]
    return cropped_datas

class ToTensor(torch.nn.Module):
  def forward(self, datas):
    tensor = torch.from_numpy(datas.astype(np.float32)) #transform datas into tensor
    return tensor
#composition of the transformation to apply
transforms = Compose([
    CropTransform(),
    ToTensor()
])

**Datas uploading from Google Drive**

In [4]:
#define the type of set poseplan, those values are the used inside the ORION architecture
num_classes=10
num_poses=105


# initialize paths
train_dataset = ImportDatas(r'/content/drive/MyDrive/ORION_dataset/ModelNet10_12rot/poseplan_MN10_12/hdf5/train_allrot/train.hdf5.txt', transform=transforms)
validation_dataset = ImportDatas(r'/content/drive/MyDrive/ORION_dataset/ModelNet10_12rot/poseplan_MN10_12/hdf5/validation_allrot/train.hdf5.txt', transform=transforms)
test_dataset = ImportDatas(r'/content/drive/MyDrive/ORION_dataset/ModelNet10_12rot/poseplan_MN10_12/hdf5/test_allrot/train.hdf5.txt', transform=transforms)

# load datas
#change batch size for validation and test. 12 if dealing with 12 rotations, 24 for 24 rotations
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2, drop_last=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=12, shuffle=False, num_workers=2, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=12, shuffle=False, num_workers=2)

#**Print one object with label**
We used this code in order to visualize the dataset composition. It's not necessary to train and test a network.


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
iterator = tqdm(validation_dataloader, disable=True)
torch.set_printoptions(threshold=327680)
for datas, label_class, label_pose in iterator:
        datas = datas.squeeze().to(device)
        label_class = label_class.squeeze().to(device)
        label_pose = label_pose.squeeze().to(device)
        #print("Batch Data Shape:", datas)
        #print("Batch Class Labels Shape:", label_class.shape)
        #print("Batch Pose Labels Shape:", label_pose.shape)
        #print(label_class)
        print(label_pose)

#print("Length of Train Dataset:", len(train_dataset))
#print("Length of Validation Dataset:", len(validation_dataset))
#print("Length of Test Dataset:", len(test_dataset))

# **ORION nets**
**Definition of the ORION net with the same architecture provided into the paper**

In [5]:
from torch.nn import Module, Sequential, Conv3d, BatchNorm3d, ReLU, Dropout3d, MaxPool3d, Linear, LeakyReLU, Dropout, ELU

# Define a simple CNN architecture
class ORION(Module):
    def __init__(self, num_classes, num_poses):
        super().__init__()
        self.model = Sequential(
          # Definition of Conv1
          Conv3d(in_channels=1, out_channels=32, kernel_size=5, stride=2),
          BatchNorm3d(num_features=32),
          ReLU(),
          Dropout3d(p=0.2),

          # Definition of Conv2
          Conv3d(in_channels=32, out_channels=64, kernel_size=3, stride=1),
          BatchNorm3d(num_features=64),
          ReLU(),
          MaxPool3d(kernel_size=2, stride=2),
          Dropout3d(p=0.3),
          )
        #fc layer
        self.fc1 = Sequential(
            Linear(64*6*6*6, out_features=128),
            ReLU(),
            Dropout(p=0.4)
        )
        #classification of class and pose
        self.class_layer = Linear(128, num_classes)
        self.pose_layer = Linear(128, num_poses)
        self.apply(self._init_weights)



    def forward(self, x):
        x = self.model(x).reshape((x.shape[0], -1))
        x = self.fc1(x)
        class_output = self.class_layer(x)
        pose_output = self.pose_layer(x)
        return class_output, pose_output

    def _init_weights(self, module):
        if isinstance(module, torch.nn.Linear):
            torch.nn.init.normal_(module.weight, std=0.01)
            if module.bias is not None:
                module.bias.data.zero_()
        if isinstance(module, torch.nn.Conv3d):
            torch.nn.init.kaiming_normal_(module.weight, nonlinearity='relu')
            if module.bias is not None:
                module.bias.data.zero_()
        torch.nn.init.normal_(self.class_layer.weight, std=0.01)
        torch.nn.init.normal_(self.pose_layer.weight, std=0.01)

        if self.class_layer.bias is not None:
            self.class_layer.bias.data.zero_()
        if self.pose_layer.bias is not None:
            self.pose_layer.bias.data.zero_()

**Modified versions of the ORION net**

**Extended ORION (from Paper)**

In [6]:
# Define a simple CNN architecture
class EXTENDED_ORION(Module):
    def __init__(self, num_classes, num_poses):
        super().__init__()
        self.model = Sequential(
          # Definition of Conv1
          Conv3d(in_channels=1, out_channels=32, kernel_size=3, stride=2),
          BatchNorm3d(num_features=32),
          ReLU(),
          Dropout3d(p=0.2),

          # Definition of Conv2
          Conv3d(in_channels=32, out_channels=64, kernel_size=3, stride=1),
          BatchNorm3d(num_features=64),
          ReLU(),
          Dropout3d(p=0.3),

          # Definition of Conv3
          Conv3d(in_channels=64, out_channels=128, kernel_size=3, stride=1),
          BatchNorm3d(num_features=128),
          ReLU(),
          Dropout3d(p=0.4),

          # Definition of Conv4
          Conv3d(in_channels=128, out_channels=256, kernel_size=3, stride=1),
          BatchNorm3d(num_features=256),
          ReLU(),
          MaxPool3d(kernel_size=2, stride=2),
          Dropout3d(p=0.6),
          )

        self.fc1 = Sequential(
            Linear(256*4*4*4, out_features=128),
            ReLU(),
            Dropout(p=0.4)
        )
        self.class_layer = Linear(128, num_classes)
        self.pose_layer = Linear(128, num_poses)
        self.apply(self._init_weights)



    def forward(self, x):
        x = self.model(x).reshape((x.shape[0], -1))
        x = self.fc1(x)
        class_output = self.class_layer(x)
        pose_output = self.pose_layer(x)
        return class_output, pose_output

    def _init_weights(self, module):
        if isinstance(module, torch.nn.Linear):
            torch.nn.init.normal_(module.weight, std=0.01)
            if module.bias is not None:
                module.bias.data.zero_()
        if isinstance(module, torch.nn.Conv3d):
            torch.nn.init.kaiming_normal_(module.weight, nonlinearity='relu')
            if module.bias is not None:
                module.bias.data.zero_()
        torch.nn.init.normal_(self.class_layer.weight, std=0.01)
        torch.nn.init.normal_(self.pose_layer.weight, std=0.01)

        if self.class_layer.bias is not None:
            self.class_layer.bias.data.zero_()

        if self.pose_layer.bias is not None:
            self.pose_layer.bias.data.zero_()

**Using Leaky Relu as activation function**

In [7]:
# Define a simple CNN architecture
class ORION_LRELU(Module):
    def __init__(self, num_classes, num_poses):
        super().__init__()
        self.model = Sequential(
          # Definition of Conv1
          Conv3d(in_channels=1, out_channels=32, kernel_size=5, stride=2),
          BatchNorm3d(num_features=32),
          LeakyReLU(negative_slope=0.1),
          Dropout3d(p=0.2),

          # Definition of Conv2
          Conv3d(in_channels=32, out_channels=64, kernel_size=3, stride=1),
          BatchNorm3d(num_features=64),
          LeakyReLU(negative_slope=0.1),
          MaxPool3d(kernel_size=2, stride=2),
          Dropout3d(p=0.3),
          )

        self.fc1 = Sequential(
            Linear(64*6*6*6, out_features=128),
            ReLU(),
            Dropout(p=0.4)
        )
        self.class_layer = Linear(128, num_classes)
        self.pose_layer = Linear(128, num_poses)
        self.apply(self._init_weights)



    def forward(self, x):
        x = self.model(x).reshape((x.shape[0], -1))
        x = self.fc1(x)
        class_output = self.class_layer(x)
        pose_output = self.pose_layer(x)
        return class_output, pose_output

    def _init_weights(self, module):
        if isinstance(module, torch.nn.Linear):
            torch.nn.init.normal_(module.weight, std=0.01)
            if module.bias is not None:
                module.bias.data.zero_()
        if isinstance(module, torch.nn.Conv3d):
            torch.nn.init.kaiming_normal_(module.weight, nonlinearity='relu')
            if module.bias is not None:
                module.bias.data.zero_()
        torch.nn.init.normal_(self.class_layer.weight, std=0.01)
        torch.nn.init.normal_(self.pose_layer.weight, std=0.01)

        if self.class_layer.bias is not None:
            self.class_layer.bias.data.zero_()

        if self.pose_layer.bias is not None:
            self.pose_layer.bias.data.zero_()

In [8]:
# Define a simple CNN architecture
class EXTENDED_ORION_LRELU(Module):
    def __init__(self, num_classes, num_poses):
        super().__init__()
        self.model = Sequential(
          # Definition of Conv1
          Conv3d(in_channels=1, out_channels=32, kernel_size=3, stride=2),
          BatchNorm3d(num_features=32),
          LeakyReLU(negative_slope=0.1),
          Dropout3d(p=0.2),

          # Definition of Conv2
          Conv3d(in_channels=32, out_channels=64, kernel_size=3, stride=1),
          BatchNorm3d(num_features=64),
          LeakyReLU(negative_slope=0.1),
          Dropout3d(p=0.3),

          # Definition of Conv3
          Conv3d(in_channels=64, out_channels=128, kernel_size=3, stride=1),
          BatchNorm3d(num_features=128),
          LeakyReLU(negative_slope=0.1),
          Dropout3d(p=0.4),

          # Definition of Conv4
          Conv3d(in_channels=128, out_channels=256, kernel_size=3, stride=1),
          BatchNorm3d(num_features=256),
          LeakyReLU(negative_slope=0.1),
          MaxPool3d(kernel_size=2, stride=2),
          Dropout3d(p=0.6),
          )

        self.fc1 = Sequential(
            Linear(256*4*4*4, out_features=128),
            ReLU(),
            Dropout(p=0.4)
        )
        self.class_layer = Linear(128, num_classes)
        self.pose_layer = Linear(128, num_poses)
        self.apply(self._init_weights)



    def forward(self, x):
        x = self.model(x).reshape((x.shape[0], -1))
        x = self.fc1(x)
        class_output = self.class_layer(x)
        pose_output = self.pose_layer(x)
        return class_output, pose_output

    def _init_weights(self, module):
        if isinstance(module, torch.nn.Linear):
            torch.nn.init.normal_(module.weight, std=0.01)
            if module.bias is not None:
                module.bias.data.zero_()
        if isinstance(module, torch.nn.Conv3d):
            torch.nn.init.kaiming_normal_(module.weight, nonlinearity='relu')
            if module.bias is not None:
                module.bias.data.zero_()
        torch.nn.init.normal_(self.class_layer.weight, std=0.01)
        torch.nn.init.normal_(self.pose_layer.weight, std=0.01)

        if self.class_layer.bias is not None:
            self.class_layer.bias.data.zero_()

        if self.pose_layer.bias is not None:
            self.pose_layer.bias.data.zero_()


**Using Exponential- linear unit as activation function**

In [9]:
# Define a simple CNN architecture
class ORION_ELU(Module):
    def __init__(self, num_classes, num_poses):
        super().__init__()
        self.model = Sequential(
          # Definition of Conv1
          Conv3d(in_channels=1, out_channels=32, kernel_size=5, stride=2),
          BatchNorm3d(num_features=32),
          ELU(),
          Dropout3d(p=0.2),

          # Definition of Conv2
          Conv3d(in_channels=32, out_channels=64, kernel_size=3, stride=1),
          BatchNorm3d(num_features=64),
          ELU(),
          MaxPool3d(kernel_size=2, stride=2),
          Dropout3d(p=0.3),
          )

        self.fc1 = Sequential(
            Linear(64*6*6*6, out_features=128),
            ReLU(),
            Dropout(p=0.4)
        )
        self.class_layer = Linear(128, num_classes)
        self.pose_layer = Linear(128, num_poses)
        self.apply(self._init_weights)



    def forward(self, x):
        x = self.model(x).reshape((x.shape[0], -1))
        x = self.fc1(x)
        class_output = self.class_layer(x)
        pose_output = self.pose_layer(x)
        return class_output, pose_output

    def _init_weights(self, module):
        if isinstance(module, torch.nn.Linear):
            torch.nn.init.normal_(module.weight, std=0.01)
            if module.bias is not None:
                module.bias.data.zero_()
        if isinstance(module, torch.nn.Conv3d):
            torch.nn.init.kaiming_normal_(module.weight, nonlinearity='relu')
            if module.bias is not None:
                module.bias.data.zero_()
        torch.nn.init.normal_(self.class_layer.weight, std=0.01)
        torch.nn.init.normal_(self.pose_layer.weight, std=0.01)

        if self.class_layer.bias is not None:
            self.class_layer.bias.data.zero_()

        if self.pose_layer.bias is not None:
            self.pose_layer.bias.data.zero_()

In [10]:
# Define a simple CNN architecture
class EXTENDED_ORION_ELU(Module):
    def __init__(self, num_classes, num_poses):
        super().__init__()
        self.model = Sequential(
          # Definition of Conv1
          Conv3d(in_channels=1, out_channels=32, kernel_size=3, stride=2),
          BatchNorm3d(num_features=32),
          ELU(),
          Dropout3d(p=0.2),

          # Definition of Conv2
          Conv3d(in_channels=32, out_channels=64, kernel_size=3, stride=1),
          BatchNorm3d(num_features=64),
          ELU(),
          Dropout3d(p=0.3),

          # Definition of Conv3
          Conv3d(in_channels=64, out_channels=128, kernel_size=3, stride=1),
          BatchNorm3d(num_features=128),
          ELU(),
          Dropout3d(p=0.4),

          # Definition of Conv4
          Conv3d(in_channels=128, out_channels=256, kernel_size=3, stride=1),
          BatchNorm3d(num_features=256),
          ELU(),
          MaxPool3d(kernel_size=2, stride=2),
          Dropout3d(p=0.6),
          )

        self.fc1 = Sequential(
            Linear(256*4*4*4, out_features=128),
            ReLU(),
            Dropout(p=0.4)
        )
        self.class_layer = Linear(128, num_classes)
        self.pose_layer = Linear(128, num_poses)
        self.apply(self._init_weights)



    def forward(self, x):
        x = self.model(x).reshape((x.shape[0], -1))
        x = self.fc1(x)
        class_output = self.class_layer(x)
        pose_output = self.pose_layer(x)
        return class_output, pose_output

    def _init_weights(self, module):
        if isinstance(module, torch.nn.Linear):
            torch.nn.init.normal_(module.weight, std=0.01)
            if module.bias is not None:
                module.bias.data.zero_()
        if isinstance(module, torch.nn.Conv3d):
            torch.nn.init.kaiming_normal_(module.weight, nonlinearity='relu')
            if module.bias is not None:
                module.bias.data.zero_()
        torch.nn.init.normal_(self.class_layer.weight, std=0.01)
        torch.nn.init.normal_(self.pose_layer.weight, std=0.01)

        if self.class_layer.bias is not None:
            self.class_layer.bias.data.zero_()

        if self.pose_layer.bias is not None:
            self.pose_layer.bias.data.zero_()

**Only classes evaluation**

In [11]:
# Define a simple CNN architecture
class ORION_OC(Module):
    def __init__(self, num_classes):
        super().__init__()
        self.model = Sequential(
          # Definition of Conv1
          Conv3d(in_channels=1, out_channels=32, kernel_size=5, stride=2),
          BatchNorm3d(num_features=32),
          ReLU(),
          Dropout3d(p=0.2),

          # Definition of Conv2
          Conv3d(in_channels=32, out_channels=64, kernel_size=3, stride=1),
          BatchNorm3d(num_features=64),
          ReLU(),
          MaxPool3d(kernel_size=2, stride=2),
          Dropout3d(p=0.3),
          )

        self.fc1 = Sequential(
            Linear(64*6*6*6, out_features=128),
            ReLU(),
            Dropout(p=0.4)
        )
        self.class_layer = Linear(128, num_classes)
        self.apply(self._init_weights)



    def forward(self, x):
        x = self.model(x).reshape((x.shape[0], -1))
        x = self.fc1(x)
        class_output = self.class_layer(x)
        return class_output

    def _init_weights(self, module):
        if isinstance(module, torch.nn.Linear):
            torch.nn.init.normal_(module.weight, std=0.01)
            if module.bias is not None:
                module.bias.data.zero_()
        if isinstance(module, torch.nn.Conv3d):
            torch.nn.init.kaiming_normal_(module.weight, nonlinearity='relu')
            if module.bias is not None:
                module.bias.data.zero_()
        torch.nn.init.normal_(self.class_layer.weight, std=0.01)

        if self.class_layer.bias is not None:
            self.class_layer.bias.data.zero_()

In [12]:
# Define a simple CNN architecture
class EXTENDED_ORION_OC(Module):
    def __init__(self, num_classes):
        super().__init__()
        self.model = Sequential(
          # Definition of Conv1
          Conv3d(in_channels=1, out_channels=32, kernel_size=3, stride=2),
          BatchNorm3d(num_features=32),
          ReLU(),
          Dropout3d(p=0.2),

          # Definition of Conv2
          Conv3d(in_channels=32, out_channels=64, kernel_size=3, stride=1),
          BatchNorm3d(num_features=64),
          ReLU(),
          Dropout3d(p=0.3),

          # Definition of Conv3
          Conv3d(in_channels=64, out_channels=128, kernel_size=3, stride=1),
          BatchNorm3d(num_features=128),
          ReLU(),
          Dropout3d(p=0.4),

          # Definition of Conv4
          Conv3d(in_channels=128, out_channels=256, kernel_size=3, stride=1),
          BatchNorm3d(num_features=256),
          ReLU(),
          MaxPool3d(kernel_size=2, stride=2),
          Dropout3d(p=0.6),
          )

        self.fc1 = Sequential(
            Linear(256*4*4*4, out_features=128),
            ReLU(),
            Dropout(p=0.4)
        )
        self.class_layer = Linear(128, num_classes)
        self.apply(self._init_weights)



    def forward(self, x):
        x = self.model(x).reshape((x.shape[0], -1))
        x = self.fc1(x)
        class_output = self.class_layer(x)
        return class_output

    def _init_weights(self, module):
        if isinstance(module, torch.nn.Linear):
            torch.nn.init.normal_(module.weight, std=0.01)
            if module.bias is not None:
                module.bias.data.zero_()
        if isinstance(module, torch.nn.Conv3d):
            torch.nn.init.kaiming_normal_(module.weight, nonlinearity='relu')
            if module.bias is not None:
                module.bias.data.zero_()
        torch.nn.init.normal_(self.class_layer.weight, std=0.01)

        if self.class_layer.bias is not None:
            self.class_layer.bias.data.zero_()

#**Performances evaluation functions**
**Accuracies function**

In [13]:
def accuracy (network, dataloader):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    network.to(device)
    softmax= torch.nn.Softmax(dim=1)

    iterator = tqdm(dataloader, disable=True)
    with torch.no_grad():
      #initialize empty arrays
      predicted_class_array = []
      true_class_array = []
      predicted_pose_array = []
      true_pose_array = []

      # iterating through the batches
      for datas, label_class, label_pose in iterator:
        datas = datas.to(device)
        label_class = label_class.squeeze().to(device) #remove the dimesions of size 1
        label_pose = label_pose.squeeze().to(device)
        network.eval()
        # forward pass
        predicted_class, predicted_pose = network(datas)

        # obtain the class from the output
        sum_class_pred = torch.sum(predicted_class, dim=0)
        predicted_class = torch.argmax(sum_class_pred)
        # obtain the pose label from the output
        predicted_pose = torch.argmax(predicted_pose, dim=1)
        #update the arrays concatenating the results and the true labels
        predicted_class_array.append(predicted_class)
        true_class_array.append(label_class[0]) #only the first element is important, the others are just repetitions due to the fact that each batch is composed by rotations of the same object
        predicted_pose_array.append(predicted_pose)
        true_pose_array.append(label_pose) #in this case all the labels are important

      #build a single vector
      true_class_array= torch.stack(true_class_array)
      predicted_class_array= torch.stack(predicted_class_array)
      true_pose_array = torch.cat(true_pose_array, axis=0)
      predicted_pose_array= torch.cat(predicted_pose_array, dim=0)

      # calculate accuracy
      class_accuracy = torch.sum(predicted_class_array == true_class_array)/len(true_class_array)
      pose_accuracy = torch.sum(predicted_pose_array == true_pose_array)/len(true_pose_array)

      #return all the obtained elements
      return class_accuracy, pose_accuracy


**Losses function**

In [14]:
def loss (network, dataloader):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    network.to(device)
    # defining the loss functions, creoss-entropy losses are chosen like in the paper
    loss_fn_class = CrossEntropyLoss()
    loss_fn_pose = CrossEntropyLoss()


    iterator = tqdm(dataloader, disable=True)
    with torch.no_grad():
      predicted_class_array = []
      true_class_array = []
      predicted_pose_array = []
      true_pose_array = []

      # iterating through the batches
      for datas, label_class, label_pose in iterator:
        datas = datas.to(device)
        label_class = label_class.squeeze().to(device) #remove the dimesions of size 1
        label_pose = label_pose.squeeze().to(device)
        network.eval()
        # forward pass
        predicted_class, predicted_pose = network(datas)

        predicted_class_array.append(predicted_class)
        true_class_array.append(label_class) #only the first element is import, the others are just repetions due to the fact that each batch is composed by rotations of the same object
        predicted_pose_array.append(predicted_pose)
        true_pose_array.append(label_pose) #in this case all the labels are important

      #build a single vector
      true_class_array= torch.cat(true_class_array, axis=0)
      predicted_class_array= torch.cat(predicted_class_array, axis=0)
      true_pose_array = torch.cat(true_pose_array, axis=0)
      predicted_pose_array = torch.cat(predicted_pose_array, axis=0)

      #calculate losses
      pose_loss = loss_fn_pose(predicted_pose_array, true_pose_array)
      class_loss = loss_fn_class(predicted_class_array, true_class_array)
      total_loss = (class_loss + pose_loss)/2 #weighted sum as defined in the paper

      #return all the obtained elements
      return class_loss, pose_loss, total_loss

**Accuracies function for class classificatio only**
Here we don't use the pose information.

In [15]:
def accuracy_OC (network, dataloader):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    network.to(device)
    softmax= torch.nn.Softmax(dim=1)

    iterator = tqdm(dataloader, disable=True)
    with torch.no_grad():
      predicted_class_array = []
      true_class_array = []
      # iterating through the batches
      for datas, label_class, label_pose in iterator:
        datas = datas.to(device)
        label_class = label_class.squeeze().to(device) #remove the dimesions of size 1
        label_pose = label_pose.squeeze().to(device)
        network.eval()
        # forward pass
        predicted_class = network(datas)

        # obtain the class from the output
        sum_class_pred = torch.sum(predicted_class, dim=0)
        predicted_class = torch.argmax(sum_class_pred)
        #update the arrays concatenating the results and the true labels
        predicted_class_array.append(predicted_class)
        true_class_array.append(label_class[0]) #only the first element is important, the others are just repetitions due to the fact that each batch is composed by rotations of the same object

      #build a single vector
      true_class_array= torch.stack(true_class_array)
      predicted_class_array= torch.stack(predicted_class_array)

      # calculate accuracy
      class_accuracy = torch.sum(predicted_class_array == true_class_array)/len(true_class_array)

      #return all the obtained elements
      return class_accuracy


**Losses function for class classification only**
Here we don't use the pose information.

In [16]:
def loss_OC (network, dataloader):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    network.to(device)
    # defining the loss functions, creoss-entropy losses are chosen like in the paper
    loss_fn_class = CrossEntropyLoss()

    iterator = tqdm(dataloader, disable=True)
    with torch.no_grad():
      predicted_class_array = []
      true_class_array = []

      # iterating through the batches
      for datas, label_class, label_pose in iterator:
        datas = datas.to(device)
        label_class = label_class.squeeze().to(device) #remove the dimesions of size 1
        label_pose = label_pose.squeeze().to(device)
        network.eval()
        # forward pass
        predicted_class = network(datas)

        predicted_class_array.append(predicted_class)
        true_class_array.append(label_class) #only the first element is import, the others are just repetions due to the fact that each batch is composed by rotations of the same object

      #build a single vector
      true_class_array= torch.cat(true_class_array, axis=0)
      predicted_class_array= torch.cat(predicted_class_array, axis=0)

      #calculate losses
      class_loss = loss_fn_class(predicted_class_array, true_class_array)

      #return all the obtained elements
      return class_loss

#**Data logging**
This section is needed in order to acquire graphical datas. It's not needed to train and test a network. However the run is mandatory in our case since wandb is also implemented into the training process. If it's not made the training process give an error.

In [17]:
# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="Wandb_accuracy_SDGvsAdam",

    # track hyperparameters and run metadata
    config={
    "dataset": "ModelNet10",
    "epochs": 200,
    }
)
wandb.define_metric("epoch")

wandb.define_metric("validation_loss_SDG", step_metric="epoch")
wandb.define_metric("class_validation_loss_SDG", step_metric="epoch")
wandb.define_metric("pose_validation_loss_SDG", step_metric="epoch")
wandb.define_metric("class_validation_accuracy_SDG", step_metric="epoch")
wandb.define_metric("pose_validation_accuracy_SDG", step_metric="epoch")
wandb.define_metric("class_validation_error_SDG", step_metric="epoch")
wandb.define_metric("pose_validation_error_SDG", step_metric="epoch")


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


**Run this to deal only with classes and not pose**

In [ ]:
# start a new wandb
wandb.init(

    project="Modelnet10_onlyclassesSDG",

    config={
    "dataset": "ModelNet10",
    "epochs": 200,
    }
)
wandb.define_metric("epoch")
wandb.define_metric("class_validation_loss", step_metric="epoch")
wandb.define_metric("class_validation_accuracy", step_metric="epoch")
wandb.define_metric("class_validation_error", step_metric="epoch")

# **Training process**

In [18]:
def training(model,train_data_loading, validation_data_loading, epochs, opt, early_stopping):

  device = "cuda" if torch.cuda.is_available() else "cpu"
  model.to(device)
  best_loss=np.inf #set the best value of the loss to infinite
  early_stopping_val=0
  class_loss_function = CrossEntropyLoss()
  pose_loss_function = CrossEntropyLoss()

  #start loop for each epoch, defined in input
  for epoch in range(epochs):

    print(f"Epoch: {epoch+1}")
    model.train()
    elements = tqdm(train_data_loading)
    for datas, label_class, label_pose in elements:
      #input real labels
      datas=datas.to(device)
      label_class=label_class.squeeze().to(device)
      label_pose=label_pose.squeeze().to(device)
      #do the forward pass
      label_class_prediction, label_pose_prediction=model(datas)

      #calculate losses
      class_loss = class_loss_function(label_class_prediction, label_class)
      pose_loss = pose_loss_function(label_pose_prediction, label_pose)
      train_loss = (class_loss + pose_loss)/2
      # logging training losses
      wandb.log({"train_loss": train_loss, "pose_train_loss": pose_loss, "class_train_loss": class_loss })

      #backword pass
      opt.zero_grad()
      train_loss.backward()
      opt.step()
      #view batch results
      elements.set_description(f"Train loss: {train_loss.detach().cpu().numpy()} class loss: {class_loss.detach().cpu().numpy()}  pose loss: {pose_loss.detach().cpu().numpy()}")
    #compute accuracies and losses on validation set
    class_accuracy_val, pose_accuracy_val = accuracy(model,validation_data_loading)
    class_loss_val, pose_loss_val, total_loss_val=loss(model, validation_data_loading)

    # logging validation results
    wandb.log({"epoch": epoch+1,
                "validation_loss": total_loss_val,
                "pose_validation_loss": pose_loss_val,
                "class_validation_loss": class_loss_val,
                "class_validation_accuracy": class_accuracy_val,
                "pose_validation_accuracy": pose_accuracy_val,
                "class_validation_error": 1-class_accuracy_val,
                "pose_validation_error": 1-pose_accuracy_val})

    #display elements
    print(f"class accuracy: {class_accuracy_val}, \n pose accuracy: {pose_accuracy_val}")
    print(f"class loss: {class_loss_val}, \n pose loss: {pose_loss} , \n total loss: {total_loss_val}")
    #updating model looking at the loss
    if total_loss_val < best_loss:
        print("Saved Update Model")
        #print(model)
        torch.save(model.state_dict(), "model.pt")
        best_loss = total_loss_val
        early_stopping_val=0 #re-initialize the count
    else:
        early_stopping_val+=1
        #check early stopping and exit training loop
    if early_stopping_val >= early_stopping :
      print("EARLY STOPPING")
      break

**Define a training process for the usage of classes only, without considering the orientation**

In [19]:
def training_OC(model,train_data_loading, validation_data_loading, epochs, opt, early_stopping):

  device = "cuda" if torch.cuda.is_available() else "cpu"
  model.to(device)
  best_loss=np.inf #set the best value of the loss to infinite
  early_stopping_val=0
  class_loss_function = CrossEntropyLoss()

  #start loop for each epoch, defined in input
  for epoch in range(epochs):

    print(f"Epoch: {epoch+1}")
    model.train()
    elements = tqdm(train_data_loading)
    for datas, label_class, label_pose in elements:
      #input real labels
      datas=datas.to(device)
      label_class=label_class.squeeze().to(device)
      label_pose=label_pose.squeeze().to(device)
      #do the forward pass
      label_class_prediction=model(datas)

      #calculate losses
      class_loss = class_loss_function(label_class_prediction, label_class)


      #backword pass
      opt.zero_grad()
      class_loss.backward()
      opt.step()
      #view batch results
      elements.set_description(f"class loss: {class_loss.detach().cpu().numpy()}")
    #compute accuracies and losses on validation set
    class_accuracy_val = accuracy_OC(model,validation_data_loading)
    class_loss_val = loss_OC(model, validation_data_loading)
    # logging validation results
    wandb.log({"epoch": epoch+1,
                "class_validation_loss": class_loss_val,
                "class_validation_accuracy": class_accuracy_val,
                "class_validation_error": 1-class_accuracy_val})

    #display elements
    print(f"class accuracy: {class_accuracy_val}")
    print(f"class loss: {class_loss_val}")
    #updating model looking at the loss
    if class_loss_val < best_loss:
        print("Saved Model")
        torch.save(model.state_dict(), "model.pt")
        best_loss = class_loss_val
        early_stopping_val=0 #re-initialize the count
    else:
        early_stopping_val+=1
        #check early stopping and exit training loop
    if early_stopping_val >= early_stopping :
      print("EARLY STOPPING")
      break

# **Initialization and running**

In [20]:
from torch.optim import SGD, Adam

#manually define the architecture type
chosen_model=1;
#manually define the optimizer type
chosen_opt=1;

#define epochs and early stopping
epochs=200
es=20

#ReLU
if chosen_model==1 and chosen_opt==1:
  model=ORION(num_classes, num_poses)
  opt = SGD(model.parameters(), lr=1e-3, momentum=0.9, nesterov=True)
  training(model, train_dataloader, validation_dataloader, epochs, opt, es)
elif chosen_model==1 and chosen_opt==2:
  model=ORION(num_classes, num_poses)
  opt = Adam(model.parameters(), lr=1e-4)
  training(model, train_dataloader, validation_dataloader, epochs, opt, es)
elif chosen_model==2 and chosen_opt==1:
  model=EXTENDED_ORION(num_classes, num_poses)
  opt = SGD(model.parameters(), lr=1e-3, momentum=0.9, nesterov=True)
  training(model, train_dataloader, validation_dataloader, epochs, opt, es)
elif chosen_model==2 and chosen_opt==2:
  model=EXTENDED_ORION(num_classes, num_poses)
  opt = Adam(model.parameters(), lr=1e-4)
  training(model, train_dataloader, validation_dataloader, epochs, opt, es)

#Leaky Relu
elif chosen_model==3 and chosen_opt==1:
  model=ORION_LRELU(num_classes, num_poses)
  opt = SGD(model.parameters(), lr=1e-3, momentum=0.9, nesterov=True)
  training(model, train_dataloader, validation_dataloader, epochs, opt, es)
elif chosen_model==3 and chosen_opt==2:
  model=ORION_LRELU(num_classes, num_poses)
  opt = Adam(model.parameters(), lr=1e-4)
  training(model, train_dataloader, validation_dataloader, epochs, opt, es)
elif chosen_model==4 and chosen_opt==1:
  model=EXTENDED_ORION_LRELU(num_classes, num_poses)
  opt = SGD(model.parameters(), lr=1e-3, momentum=0.9, nesterov=True)
  training(model, train_dataloader, validation_dataloader, epochs, opt, es)
elif chosen_model==4 and chosen_opt==2:
  model=EXTENDED_ORION_LRELU(num_classes, num_poses)
  opt = Adam(model.parameters(), lr=1e-4)
  training(model, train_dataloader, validation_dataloader, epochs, opt, es)

#ELU
elif chosen_model==5 and chosen_opt==1:
  model=ORION_ELU(num_classes, num_poses)
  opt = SGD(model.parameters(), lr=1e-3, momentum=0.9, nesterov=True)
  training(model, train_dataloader, validation_dataloader, epochs, opt, es)
elif chosen_model==5 and chosen_opt==2:
  model=ORION_ELU(num_classes, num_poses)
  opt = Adam(model.parameters(), lr=1e-4)
  training(model, train_dataloader, validation_dataloader, epochs, opt, es)
elif chosen_model==6 and chosen_opt==1:
  model=EXTENDED_ORION_ELU(num_classes, num_poses)
  opt = SGD(model.parameters(), lr=1e-3, momentum=0.9, nesterov=True)
  training(model, train_dataloader, validation_dataloader, epochs, opt, es)
elif chosen_model==6 and chosen_opt==2:
  model=EXTENDED_ORION_ELU(num_classes, num_poses)
  opt = Adam(model.parameters(), lr=1e-4)
  training(model, train_dataloader, validation_dataloader, epochs, opt, es)

#Only classes evalyuation
elif chosen_model==7 and chosen_opt==1:
  model=ORION_OC(num_classes)
  opt = SGD(model.parameters(), lr=1e-3, momentum=0.9, nesterov=True) #validation accuracy 94.67% test 90.75%
  training_OC(model, train_dataloader, validation_dataloader, epochs, opt, es)
elif chosen_model==7 and chosen_opt==2:
  model=ORION_OC(num_classes)
  opt = Adam(model.parameters(), lr=1e-4) #validation accuracy 93.33% test 90.86%
  training_OC(model, train_dataloader, validation_dataloader, epochs, opt, es)
elif chosen_model==8 and chosen_opt==1:
  model=EXTENDED_ORION_OC(num_classes)
  opt = SGD(model.parameters(), lr=1e-3, momentum=0.9, nesterov=True)
  training_OC(model, train_dataloader, validation_dataloader, epochs, opt, es)
elif chosen_model==8 and chosen_opt==2:
  model=EXTENDED_ORION_OC(num_classes)
  opt = Adam(model.parameters(), lr=1e-4)
  training_OC(model, train_dataloader, validation_dataloader, epochs, opt, es)



else:
  print("Indeces are wrong")


wandb.finish()

Epoch: 1


Train loss: 0.7249283194541931 class loss: 0.49175459146499634  pose loss: 0.9581020474433899: 100%|██████████| 1371/1371 [00:36<00:00, 37.08it/s]


class accuracy: 0.8592493534088135, 
 pose accuracy: 0.6902368068695068
class loss: 0.5240792632102966, 
 pose loss: 0.9581020474433899 , 
 total loss: 0.8158986568450928
Saved Update Model
Epoch: 2


Train loss: 0.37988850474357605 class loss: 0.237222358584404  pose loss: 0.5225546360015869: 100%|██████████| 1371/1371 [00:34<00:00, 40.08it/s]


class accuracy: 0.8686327338218689, 
 pose accuracy: 0.741510272026062
class loss: 0.4354113042354584, 
 pose loss: 0.5225546360015869 , 
 total loss: 0.640637218952179
Saved Update Model
Epoch: 3


...

Train loss: 0.19371318817138672 class loss: 0.14366932213306427  pose loss: 0.24375706911087036: 100%|██████████| 1371/1371 [00:34<00:00, 39.24it/s]


class accuracy: 0.9142091274261475, 
 pose accuracy: 0.8395889401435852
class loss: 0.43033191561698914, 
 pose loss: 0.24375706911087036 , 
 total loss: 0.5078661441802979
Epoch: 29


Train loss: 0.35960590839385986 class loss: 0.25200510025024414  pose loss: 0.4672067165374756: 100%|██████████| 1371/1371 [00:36<00:00, 37.65it/s]


class accuracy: 0.9021447896957397, 
 pose accuracy: 0.8316577672958374
class loss: 0.42805370688438416, 
 pose loss: 0.4672067165374756 , 
 total loss: 0.5041864514350891
EARLY STOPPING


class_train_loss,▇▅█▅▃▄▅▂▃▃▃▂▃▃▂▂▂▃▁▃▂▂▁▄▁▁▂▂▂▁▁▂▁▂▂▃▁▁▁▁
class_validation_accuracy,▁▂▄▇▆▇█▇▇▆▇▆▆█▅▇▆▇▇█▇█▇█▆▇▇█▆
class_validation_error,█▇▅▂▃▂▁▂▂▃▂▃▃▁▄▂▃▂▂▁▂▁▂▁▃▂▂▁▃
class_validation_loss,█▅▃▂▂▂▁▂▁▂▂▂▄▂▃▃▃▃▄▃▄▄▄▃▅▄▃▅▅
epoch,▁▁▁▂▂▂▃▃▃▃▃▄▄▄▅▅▅▅▅▆▆▆▇▇▇▇▇██
pose_train_loss,█▅▅▅▃▄▄▂▂▂▃▂▃▂▃▃▁▂▂▂▂▂▁▃▁▂▁▂▂▁▂▂▁▃▂▂▂▁▁▂
pose_validation_accuracy,▁▃▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇█▇▇█████████
pose_validation_error,█▆▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▂▂▁▁▁▁▁▁▁▁▁
pose_validation_loss,█▅▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss,█▅▆▅▃▄▄▂▂▂▃▂▃▃▃▂▁▂▂▂▂▂▁▄▁▁▂▂▂▁▂▂▁▃▂▂▂▁▁▂
validation_loss,█▅▃▂▂▂▁▁▁▁▁▁▂▁▂▂▂▂▂▂▂▂▂▁▂▂▁▂▂


# **Testing Phase**
This print the test and validation loss and accuracy. Notice that if the only class version is used, some little changes are needed.

In [23]:
model.load_state_dict(torch.load("model.pt"))

val_class_accuracy, val_pose_accuracy = accuracy(model, validation_dataloader)
#val_class_accuracy = accuracy_OC(model, validation_dataloader)

print("Validation Class Accuracy : "+str(val_class_accuracy))
print("Validation Pose Accuracy : "+str(val_pose_accuracy))

test_class_accuracy, test_pose_accuracy = accuracy(model, test_dataloader)
#test_class_accuracy = accuracy_OC(model, test_dataloader)


print("Test Class Accuracy : "+str(test_class_accuracy))
print("Test Pose Accuracy : "+str(test_pose_accuracy))

Validation Class Accuracy : tensor(0.9115, device='cuda:0')
Validation Pose Accuracy : tensor(0.8177, device='cuda:0')
Test Class Accuracy : tensor(0.9395, device='cuda:0')
Test Pose Accuracy : tensor(0.8340, device='cuda:0')
